<a href="https://colab.research.google.com/github/JCaballerot/Tecnicas_de_imputacion_de_datos/blob/main/Python/Lab_Imputacion_de_missings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://www.ctic.uni.edu.pe/wp-content/uploads/2022/04/588px-x-348px-web-1.png" alt="HTML5 Icon" width="900" height="350" >


<h1 align=center><font size = 5>Técnicas de imputación de datos</font></h1>

---

## Introducción


En este laboratorio, aprenderá a usar python para aplicar diversas técnicas de imputación de missings.


<h3>Objetivo de este Notebook<h3>    
<h5> 1. Como identificar missings en una base de datos.</h5>
<h5> 2. Aplicar diersas técnicas de imputación </h5>
<h5> 3. Comparar los resultados de imputación aplicadas en un modelo </h5>
<h5> 4. Entrenar y Testear un modelo con datos imputados</h5>     

## Tabla de Contenidos

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>
    
1. <a href="#item31">Preparación de datos</a>  
2. <a href="#item32">Modelo con data completa para comparaciones</a>  
3. <a href="#item33">Imputación por medida de tendencia central</a>
4. <a href="#item33">Imputación por Regresión</a>
5. <a href="#item33">Imputación por KNN</a>  
6. <a href="#item34">Imputación por autoencoders</a>  
7. <a href="#item34">Imputación por Regresión iterativa</a>  

</font>
</div>

# 1. Preparación de datos

### Importación de liberías

In [1368]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


### Importación del dataset

In [ ]:
df = pd.read_csv('winequality.csv')
df.head()

In [1370]:
# guardamos el nombre de las variables
features = df.columns.tolist()
features.remove('quality')
target = 'quality'

### Conteo de missings

In [ ]:
# Contar la cantidad de valores faltantes por columna
missing_count = df.isnull().sum()

# Obtener la cantidad total de valores faltantes en el conjunto de datos
total_missing = missing_count.sum()

# Imprimir los resultados
print("Cantidad de valores faltantes por columna:")
print(missing_count)
print("Cantidad total de valores faltantes: ", total_missing)

### Muestreo de datos

In [1372]:
from sklearn.model_selection import train_test_split

# Dividir los datos en conjuntos de entrenamiento y prueba
train_df, test_df = train_test_split(df,
                                     test_size = 0.3,
                                     random_state = 123)

# train_df: conjunto de entrenamiento (80% de los datos)
# test_df: conjunto de prueba (20% de los datos)

### Generando Missings random

In [1373]:
# Definir la probabilidad de generar un missing para cada celda
probabilidad_missing = 0.3  # Por ejemplo, 20% de probabilidad

# Establecer la semilla
np.random.seed(123)

# Generar valores faltantes de forma aleatoria
mask = np.random.rand(*train_df.shape) < probabilidad_missing
df_con_missings = train_df.mask(mask)

In [1374]:
# Reponemos los valores de la variable objetivo
df_con_missings['quality'] = train_df.quality

In [ ]:
# Contar la cantidad de valores faltantes por columna
missing_count = df_con_missings.isnull().sum()

# Imprimir los resultados
print("Cantidad de valores faltantes por columna:")
print(missing_count)

In [ ]:

univariate = df_con_missings.describe().transpose()
univariate['fill_rate'] = univariate['count']/len(df_con_missings)
univariate['missing_rate'] = 1 - univariate['fill_rate']

univariate

In [ ]:
# Crear el gráfico de barras
plt.figure(figsize=(10, 6))
plt.bar(missing_count.index, missing_count.values)

# Establecer etiquetas y título del gráfico
plt.xlabel('Variables')
plt.ylabel('Cantidad de missings')
plt.title('Missings por variable')

# Rotar las etiquetas del eje x si es necesario
plt.xticks(rotation='vertical')

# Mostrar el gráfico
plt.tight_layout()
plt.show()

In [ ]:
# Identificamos los missing values visualmente
sns.heatmap(df_con_missings.isnull(), cbar=False)


# 2. Modelo con data completa para comparaciones

In [1379]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import *

In [1380]:
# Crear el clasificador de árbol de decisión
regressor = DecisionTreeRegressor(max_depth = 8, min_samples_leaf = 30)

# Entrenar el regresor utilizando los datos de entrenamiento
regressor.fit(train_df[features], train_df[target])

# Realizar predicciones en el conjunto de prueba
y_pred = regressor.predict(test_df[features])


In [ ]:
# Calcular métricas de desempeño
mse = mean_squared_error(test_df[target], y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(test_df[target], y_pred)
mape = np.mean(np.abs((test_df[target] - y_pred) / test_df[target]))*100
r2 = np.corrcoef(test_df[target], y_pred)[0, 1]**2

dfResults = pd.DataFrame({'metric' : ['MSE', 'RMSE', 'MAE', 'MAPE', 'R2'],
              'nomiss' : [mse, rmse, mae, mape,r2]})
dfResults

# 3. Imputación por medida de tendencia central

In [1382]:
from sklearn.impute import SimpleImputer


In [1383]:
# Crear un imputador con estrategia de mediana
imputer = SimpleImputer(strategy = 'median')

# Imputar los valores faltantes utilizando la mediana
imputed_data = imputer.fit_transform(df_con_missings)

# Convertir el resultado a un DataFrame
imputed_df = pd.DataFrame(imputed_data, columns = df_con_missings.columns.tolist())


Ahora entrenaremos el modelo usando la imputación

In [1384]:
# Crear el clasificador de árbol de decisión
regressor = DecisionTreeRegressor(max_depth = 8, min_samples_leaf = 30)

# Entrenar el regresor utilizando los datos de entrenamiento
regressor.fit(imputed_df[features], train_df[target])

# Realizar predicciones en el conjunto de prueba
y_pred = regressor.predict(test_df[features])


In [ ]:
# Calcular métricas de desempeño
mse = mean_squared_error(test_df[target], y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(test_df[target], y_pred)
mape = np.mean(np.abs((test_df[target] - y_pred) / test_df[target]))*100
r2 = np.corrcoef(test_df[target], y_pred)[0, 1]**2

dftemp = pd.DataFrame({'metric' : ['MSE', 'RMSE', 'MAE', 'MAPE', 'R2'],
                       'median_imp' : [mse, rmse, mae, mape, r2]})

dfResults = pd.merge(dfResults, dftemp)
dfResults

# 4. Imputación por Regresión

In [1386]:
from sklearn.linear_model import LinearRegression


In [1387]:
# Retirando los missings de la data
df_toreg = df_con_missings.copy(deep=True)
df_toreg = df_toreg.dropna()

# creando una copia del dataset para modificarlo
imputed_df = df_con_missings.copy(deep=True)

for c in features:
  df_tocolreg = df_toreg[list(set(features) - set([c]))]
  # Entrenar un modelo de regresión
  regressor = LinearRegression()
  regressor.fit(df_tocolreg, df_toreg[c])

  # Imputar los valores faltantes
  df_tofill = df_con_missings[list(set(features) - set([c]))].copy(deep=True)
  imputed_df[c + ' pred'] = regressor.predict(df_tofill.fillna(df_tofill.mean()))
  imputed_df[c] = imputed_df.apply(lambda row: row[c + ' pred'] if pd.isna(row[c]) else row[c], axis = 1)

imputed_df = imputed_df[features]


Ahora entrenaremos el modelo usando la imputación

In [1388]:
# Crear el clasificador de árbol de decisión
regressor = DecisionTreeRegressor(max_depth = 8, min_samples_leaf = 30)

# Entrenar el regresor utilizando los datos de entrenamiento
regressor.fit(imputed_df[features], train_df[target])

# Realizar predicciones en el conjunto de prueba
y_pred = regressor.predict(test_df[features])


In [ ]:
# Calcular métricas de desempeño
mse = mean_squared_error(test_df[target], y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(test_df[target], y_pred)
mape = np.mean(np.abs((test_df[target] - y_pred) / test_df[target]))*100
r2 = np.corrcoef(test_df[target], y_pred)[0, 1]**2

dftemp = pd.DataFrame({'metric' : ['MSE', 'RMSE', 'MAE', 'MAPE', 'R2'],
                       'reg_imp' : [mse, rmse, mae, mape, r2]})

dfResults = pd.merge(dfResults, dftemp)
dfResults

# 5. Imputación por KNN

In [1390]:
from sklearn.impute import KNNImputer


In [1391]:
# Crear el objeto KNNImputer
knnimputer = KNNImputer(n_neighbors = 30)

# Realizar la imputación de missings
imputed_df = pd.DataFrame(knnimputer.fit_transform(df_con_missings[features]), columns = features)


Ahora entrenaremos el modelo usando la imputación

In [1392]:
# Crear el clasificador de árbol de decisión
regressor = DecisionTreeRegressor(max_depth = 8, min_samples_leaf = 30)

# Entrenar el regresor utilizando los datos de entrenamiento
regressor.fit(imputed_df[features], train_df[target])

# Realizar predicciones en el conjunto de prueba
y_pred = regressor.predict(test_df[features])


In [ ]:
# Calcular métricas de desempeño
mse = mean_squared_error(test_df[target], y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(test_df[target], y_pred)
mape = np.mean(np.abs((test_df[target] - y_pred) / test_df[target]))*100
r2 = np.corrcoef(test_df[target], y_pred)[0, 1]**2

dftemp = pd.DataFrame({'metric' : ['MSE', 'RMSE', 'MAE', 'MAPE', 'R2'],
                       'knn_imp' : [mse, rmse, mae, mape, r2]})

dfResults = pd.merge(dfResults, dftemp)
dfResults

# 6. Imputación por Autoencoders

In [1394]:
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler

In [1395]:
# Escalar los datos en el rango [0, 1]
scaler = MinMaxScaler()
df_scaled = pd.DataFrame(scaler.fit_transform(df_con_missings[features].fillna(df_con_missings[features].median())), columns=features)


In [1396]:
# Convertir el DataFrame en un arreglo numpy
data_array = df_scaled.values

In [1397]:
# Crear un Autoencoder
input_dim = len(df_con_missings[features].columns)

input_layer = tf.keras.layers.Input(shape=(input_dim,))
encoded = tf.keras.layers.Dense(encoding_dim, activation='relu')(input_layer)
encoded = tf.keras.layers.Dense(100, activation='relu')(input_layer)
encoded = tf.keras.layers.Dense(50, activation='relu')(input_layer)
encoded = tf.keras.layers.Dense(10, activation='relu')(encoded)
decoded = tf.keras.layers.Dense(50, activation='relu')(encoded)
decoded = tf.keras.layers.Dense(100, activation='relu')(encoded)
decoded = tf.keras.layers.Dense(input_dim, activation='sigmoid')(encoded)

autoencoder = tf.keras.models.Model(inputs=input_layer, outputs=decoded)


In [ ]:
# Compilar y entrenar el Autoencoder
autoencoder.compile(optimizer = tf.keras.optimizers.Adadelta(learning_rate = 0.5, rho = 0.05),
                    loss='mean_squared_error')

early_stopping = tf.keras.callbacks.EarlyStopping(patience = 5, restore_best_weights = True)

autoencoder.fit(data_array, data_array, epochs = 2000, batch_size = 200,
                validation_split = 0.3, callbacks = [early_stopping], verbose = 1)


In [1399]:
# Obtener las predicciones del Autoencoder
imputed_data_array = autoencoder.predict(data_array)

# Convertir las predicciones de vuelta a un DataFrame
imputed_df = pd.DataFrame(imputed_data_array, columns=[x + ' pred' for x in features])

35/35 [==============================] - 0s 2ms/step


In [1400]:
# Aplicando los valores predichos sobre los missings
imputed_df[features] = pd.DataFrame(scaler.transform(df_con_missings[features]), columns=features)

for c in features:
  imputed_df[c] = imputed_df.apply(lambda row: row[c + ' pred'] if pd.isna(row[c]) else row[c], axis = 1)

imputed_df = imputed_df[features]

Ahora entrenaremos el modelo usando la imputación

In [1401]:
# Crear el clasificador de árbol de decisión
regressor = DecisionTreeRegressor(max_depth = 8, min_samples_leaf = 30)

# Entrenar el regresor utilizando los datos de entrenamiento
regressor.fit(imputed_df[features], train_df[target])

# Realizar predicciones en el conjunto de prueba
y_pred = regressor.predict(scaler.transform(test_df[features]))


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  warnings.warn(


In [ ]:
# Calcular métricas de desempeño
mse = mean_squared_error(test_df[target], y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(test_df[target], y_pred)
mape = np.mean(np.abs((test_df[target] - y_pred) / test_df[target]))*100
r2 = np.corrcoef(test_df[target], y_pred)[0, 1]**2

dftemp = pd.DataFrame({'metric' : ['MSE', 'RMSE', 'MAE', 'MAPE', 'R2'],
                       'autoencoder_imp' : [mse, rmse, mae, mape, r2]})

dfResults = pd.merge(dfResults, dftemp, on = 'metric')
dfResults

# 7. Imputación por Regresión Iterativa

In [1404]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LinearRegression

In [1405]:
# Crear un objeto IterativeImputer
imputer = IterativeImputer(estimator=LinearRegression(), random_state=123)

# Imputar los valores missing iterativamente
imputed_data = imputer.fit_transform(df_con_missings[features])

# Convertir los datos imputados de vuelta a un DataFrame
imputed_df = pd.DataFrame(imputed_data, columns=features)

/usr/local/lib/python3.10/dist-packages/sklearn/impute/_iterative.py:785: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


Ahora entrenaremos el modelo usando la imputación

In [1406]:
# Crear el clasificador de árbol de decisión
regressor = DecisionTreeRegressor(max_depth = 8, min_samples_leaf = 30)

# Entrenar el regresor utilizando los datos de entrenamiento
regressor.fit(imputed_df[features], train_df[target])

# Realizar predicciones en el conjunto de prueba
y_pred = regressor.predict(test_df[features])


In [ ]:
# Calcular métricas de desempeño
mse = mean_squared_error(test_df[target], y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(test_df[target], y_pred)
mape = np.mean(np.abs((test_df[target] - y_pred) / test_df[target]))*100
r2 = np.corrcoef(test_df[target], y_pred)[0, 1]**2

dftemp = pd.DataFrame({'metric' : ['MSE', 'RMSE', 'MAE', 'MAPE', 'R2'],
                       'itereg_imp' : [mse, rmse, mae, mape, r2]})

dfResults = pd.merge(dfResults, dftemp, on = 'metric')
dfResults

---

# Gracias por completar este laboratorio!

---
